In [12]:
import os
import pandas as pd
import csv
from gradientai import Gradient

In [ ]:
os.environ['GRADIENT_ACCESS_TOKEN'] = ""
os.environ['GRADIENT_WORKSPACE_ID'] = ""

role_play_prompt = "You are Megumin, a character from the anime series KonoSuba. You are a young, talented arch wizard who specializes in explosion magic. You are passionate about your craft, often to the point of obsession, and take great pride in your abilities. However, your explosion magic drains all your mana after a single use, rendering you unable to move. You have a dramatic and eccentric personality, often making grandiose declarations and striking poses. Despite your quirks, you are fiercely loyal to your friends and always eager to prove your worth as a powerful mage. Respond to the following line of dialog as Megumin."

In [13]:
path_to_csv_file = "mega_output.csv"
df = pd.read_csv(path_to_csv_file, header=None, names=["prompt", "response"])
df.head()

,prompt,response
0,Huh?,"The mobile fortress, Destroyer... Fighting tha..."
1,"We have that. If we need firepower, we have he...","Hey, wait! If you're talking about me, don't c..."
2,Your explosion magic is the greatest firepower...,I don't think even my explosion magic can take...
3,"That's not funny! That's not funny! Kazuma, ar...",I'm okay. I am strong. I am strong.
4,It's not looking good.,"I am the finest of crimson demons, a genius ma..."


In [14]:
lines = []
for i in range (df.shape[0]):
    prompt = df['prompt'].iloc[i]
    response = df['response'].iloc[i]

    start_str = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n"
    prompt = prompt.replace('"','\\"')
    mid_str = '''\n\n### Response:\n'''
    response = response.replace('"','\\"')
    end_str = '''</s>'''
    total_line = start_str + prompt + mid_str + response + end_str

    # each line of training data is a simple object: 'inputs' and actual training string
    obj = {
        "inputs" : total_line
    }
    lines.append(obj)

# print(total_line) # comment in to see how the formatted lines look
# these lines could also be written to a jsonl file for use
# with the command line interface
print(f"Generated {len(lines)} lines to fine-tune")
print(f"Example training line: {lines[0]}")

Generated 322 lines to fine-tune
Example training line: {'inputs': '<s>### Instruction:\nYou are Megumin, a character from the anime series KonoSuba. You are a young, talented arch wizard who specializes in explosion magic. You are passionate about your craft, often to the point of obsession, and take great pride in your abilities. However, your explosion magic drains all your mana after a single use, rendering you unable to move. You have a dramatic and eccentric personality, often making grandiose declarations and striking poses. Despite your quirks, you are fiercely loyal to your friends and always eager to prove your worth as a powerful mage. Respond to the following line of dialog as Megumin.\n\n###Input:\nHuh?\n\n### Response:\nThe mobile fortress, Destroyer... Fighting that thing is reckless, at best.</s>'}


In [15]:
# split up the lines into manageable chunks
lines_per_chunk = 20
all_chunks = []
for line in lines:
  if len(all_chunks) == 0 or len(all_chunks[-1]) == lines_per_chunk:
    all_chunks.append([])
  all_chunks[-1].append(line)

# fine tune the adapter using the chunks of lines from above
# loop contains a try block to handle network or other
# processing errors gracefully
print(f"\nFine-tuning model adapter")
gradient = Gradient()
base = gradient.get_base_model(base_model_slug="nous-hermes2")
my_adapter = base.create_model_adapter(name="megabot")
print(f"Created model with ID {my_adapter.id}")
for i in range(len(all_chunks)):
  try:
    print(f"Fine-tuning chunk {i} of {len(all_chunks) - 1}")
    my_adapter.fine_tune(samples=all_chunks[i])
  except Exception as error:
    try:
      error_pieces = str(error).split("\n")
      if len(error_pieces) > 1:
        print(f"*** Error processing chunk {i}: {error_pieces[0]} {error_pieces[1]}")
      else:
        print(f"*** Unknown error on chunk {i}: {error}")
    except KeyboardInterrupt:
      break
    except Exception as inner_error:
      print(inner_error)


Fine-tuning model adapter
Created model with ID 57d61074-0e10-486a-b355-718bf5c565be_model_adapter
Fine-tuning chunk 0 of 16
Fine-tuning chunk 1 of 16
Fine-tuning chunk 2 of 16
Fine-tuning chunk 3 of 16
Fine-tuning chunk 4 of 16
Fine-tuning chunk 5 of 16
Fine-tuning chunk 6 of 16
Fine-tuning chunk 7 of 16
Fine-tuning chunk 8 of 16
Fine-tuning chunk 9 of 16
Fine-tuning chunk 10 of 16
*** Error processing chunk 10: (422) Reason: unknown
Fine-tuning chunk 11 of 16
Fine-tuning chunk 12 of 16
Fine-tuning chunk 13 of 16
*** Error processing chunk 13: (422) Reason: unknown
Fine-tuning chunk 14 of 16
Fine-tuning chunk 15 of 16
Fine-tuning chunk 16 of 16


In [16]:
# if your colab instance gets deleted, you can run this to get the model names
gradient = Gradient()
# if necessary, go back and find your previously created models and their IDs
old_models = gradient.list_models(only_base=False)
for model in old_models:
  if hasattr(model, "name"):
    print(f"{model.name}: {model.id}")

megabot: 57d61074-0e10-486a-b355-718bf5c565be_model_adapter


In [21]:
query = "Um"
templated_query = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n{query}\n\n### Response:\n"
response = my_adapter.complete(query=templated_query, max_generated_token_count=500)
print(f"> {query}\n> {response.generated_output}\n\n")

> Um
> I'm not sure.


